In [ ]:
# Importing Libraries 
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)
# Loading Data
stock_data_amazon= pd.read_csv('amazon_hist_6months.csv') 
stock_data_amazon['Symbol']='Amazon'
stock_data_apple= pd.read_csv('apple_hist_6months.csv')
stock_data_apple['Symbol']='Apple'
stock_data_msft= pd.read_csv('msft_hist_6months.csv')
stock_data_msft['Symbol']='Microsoft'
stock_data_tesla= pd.read_csv('tesla_hist_6months.csv')
stock_data_tesla['Symbol']='Tesla'
stock_data_ibm= pd.read_csv('ibm_hist_6months.csv')
stock_data_ibm['Symbol']='IBM'


In [ ]:
company_list=[stock_data_apple,stock_data_amazon,stock_data_ibm,stock_data_msft,stock_data_tesla]
for company in company_list:
 #removing the dollar sign and "volume column" from the stock_data
 company['Close']=company['Close/Last'].str.replace('$','')
 company.drop('Close/Last',inplace=True, axis=1)
#stock_data['Avg_Annual'] = stock_data['Avg_Annual'].astype(int)
 company ['Open'] = company ['Open'].str.replace('$', '')
 company ['High'] = company ['High'].str.replace('$', '')
 company ['Low'] = company ['Low'].str.replace('$', '')
 #setting type of fiels to float
 company['Open'] = company['Open'].astype(float)
 company ['High'] = company ['High'].astype(float)
 company ['Low'] = company ['Low'].astype(float)
 company ['Close'] = company ['Close'].astype(float)
 company['Date'] = pd.to_datetime(company['Date'])
 #print (stock_data.shape)
stock_data_tesla

,Date,Volume,Open,High,Low,Symbol,Close
0,2021-09-24,21373020,745.8900,774.8000,744.5600,Tesla,774.39
1,2021-09-23,11947530,755.0000,758.2000,747.9200,Tesla,753.64
2,2021-09-22,15126270,743.5263,753.6699,739.1200,Tesla,751.94
3,2021-09-21,16330720,734.7900,744.7399,730.4400,Tesla,739.38
4,2021-09-20,24757650,734.5577,742.0000,718.6249,Tesla,730.17
...,...,...,...,...,...,...,...
122,2021-04-01,35298380,688.3700,692.4203,659.4200,Tesla,661.75
123,2021-03-31,33337290,646.6200,672.0000,641.1100,Tesla,667.93
124,2021-03-30,39432360,601.7500,637.6600,591.0100,Tesla,635.62
125,2021-03-29,28636990,615.6400,616.4800,596.0200,Tesla,611.29


In [ ]:
#Filling missing Dates and values using the mean of previous and next day available.
company_list=[stock_data_apple,stock_data_amazon,stock_data_ibm,stock_data_msft,stock_data_tesla]
stock_list=[]
for company in company_list:
 dates = pd.date_range(start=company.Date.min(), end=company.Date.max())
 company=company.set_index('Date').reindex(dates).rename_axis('Date').reset_index()
 company['Volume']=pd.concat([company['Volume'].ffill(), company['Volume'].bfill()]).groupby(level=0).mean()
 company['Close']=pd.concat([company['Close'].ffill(), company['Close'].bfill()]).groupby(level=0).mean()
 company['Open']=pd.concat([company['Open'].ffill(), company['Open'].bfill()]).groupby(level=0).mean()
 company['High']=pd.concat([company['High'].ffill(), company['High'].bfill()]).groupby(level=0).mean()
 company['Low']=pd.concat([company['Low'].ffill(), company['Low'].bfill()]).groupby(level=0).mean()
 company['Symbol']=company['Symbol'].ffill()
 stock_list.append(company)
company

,Date,Volume,Open,High,Low,Symbol,Close
0,2021-03-26,33852830.0,641.8700,643.8200,599.8900,Tesla,618.71
1,2021-03-27,31244910.0,628.7550,630.1500,597.9550,Tesla,615.00
2,2021-03-28,31244910.0,628.7550,630.1500,597.9550,Tesla,615.00
3,2021-03-29,28636990.0,615.6400,616.4800,596.0200,Tesla,611.29
4,2021-03-30,39432360.0,601.7500,637.6600,591.0100,Tesla,635.62
...,...,...,...,...,...,...,...
178,2021-09-20,24757650.0,734.5577,742.0000,718.6249,Tesla,730.17
179,2021-09-21,16330720.0,734.7900,744.7399,730.4400,Tesla,739.38
180,2021-09-22,15126270.0,743.5263,753.6699,739.1200,Tesla,751.94
181,2021-09-23,11947530.0,755.0000,758.2000,747.9200,Tesla,753.64


In [ ]:
#combining stock data along Axis 0
stock_data_combined = pd.concat(stock_list, axis=0)
stock_data_combined.reset_index()
stock_data_combined.set_index('Date')
stock_data_combined=stock_data_combined.reset_index(drop=True)
stock_data_combined

,Date,Volume,Open,High,Low,Symbol,Close
0,2021-03-26,94071230.0,120.3500,121.4800,118.92000,Apple,121.21
1,2021-03-27,87445215.0,121.0000,122.0300,119.82495,Apple,121.30
2,2021-03-28,87445215.0,121.0000,122.0300,119.82495,Apple,121.30
3,2021-03-29,80819200.0,121.6500,122.5800,120.72990,Apple,121.39
4,2021-03-30,85671920.0,120.1100,120.4031,118.86000,Apple,119.90
...,...,...,...,...,...,...,...
910,2021-09-20,24757650.0,734.5577,742.0000,718.62490,Tesla,730.17
911,2021-09-21,16330720.0,734.7900,744.7399,730.44000,Tesla,739.38
912,2021-09-22,15126270.0,743.5263,753.6699,739.12000,Tesla,751.94
913,2021-09-23,11947530.0,755.0000,758.2000,747.92000,Tesla,753.64


Labelling of stock data

In [ ]:
#labelling of the stock data using same day approach
stock_data=stock_data_combined
list_label=[]
for i in range(len(stock_data)):
    open_price = stock_data.iloc[i]["Open"]
    close_price = stock_data.iloc[i]["Close"]
    change=round(round(close_price,4)-round(open_price,4),4)
    if int(round(np.sign(change)))==1:
     list_label.append(1)
    else:
     list_label.append(0)
stock_data['lab_sameday']=list_label
stock_data
  

,Date,Volume,Open,High,Low,Symbol,Close,lab_sameday
0,2021-03-26,94071230.0,120.3500,121.4800,118.92000,Apple,121.21,1
1,2021-03-27,87445215.0,121.0000,122.0300,119.82495,Apple,121.30,1
2,2021-03-28,87445215.0,121.0000,122.0300,119.82495,Apple,121.30,1
3,2021-03-29,80819200.0,121.6500,122.5800,120.72990,Apple,121.39,0
4,2021-03-30,85671920.0,120.1100,120.4031,118.86000,Apple,119.90,0
...,...,...,...,...,...,...,...,...
910,2021-09-20,24757650.0,734.5577,742.0000,718.62490,Tesla,730.17,0
911,2021-09-21,16330720.0,734.7900,744.7399,730.44000,Tesla,739.38,1
912,2021-09-22,15126270.0,743.5263,753.6699,739.12000,Tesla,751.94,1
913,2021-09-23,11947530.0,755.0000,758.2000,747.92000,Tesla,753.64,0


In [ ]:
#labelling of the stock data using previous day approach
list_label_1=[1]
for i in range(1,len(stock_data)):
    close_price_prev_day = stock_data.iloc[i-1]["Close"]
    close_price_curr_day = stock_data.iloc[i]["Close"]
    change=round(round(close_price_curr_day,4)-round(close_price_prev_day,4),4)
    if int(round(np.sign(change)))==1:
     list_label_1.append(1)
    else:
     list_label_1.append(0)
stock_data['lab_prevday']=list_label_1
stock_data

,Date,Volume,Open,High,Low,Symbol,Close,lab_sameday,lab_prevday
0,2021-03-26,94071230.0,120.3500,121.4800,118.92000,Apple,121.21,1,1
1,2021-03-27,87445215.0,121.0000,122.0300,119.82495,Apple,121.30,1,1
2,2021-03-28,87445215.0,121.0000,122.0300,119.82495,Apple,121.30,1,0
3,2021-03-29,80819200.0,121.6500,122.5800,120.72990,Apple,121.39,0,1
4,2021-03-30,85671920.0,120.1100,120.4031,118.86000,Apple,119.90,0,0
...,...,...,...,...,...,...,...,...,...
910,2021-09-20,24757650.0,734.5577,742.0000,718.62490,Tesla,730.17,0,0
911,2021-09-21,16330720.0,734.7900,744.7399,730.44000,Tesla,739.38,1,1
912,2021-09-22,15126270.0,743.5263,753.6699,739.12000,Tesla,751.94,1,1
913,2021-09-23,11947530.0,755.0000,758.2000,747.92000,Tesla,753.64,0,1


In [ ]:
#Merging twitter data with stock data by company name.
#read grouped csv files
twitter_NLP_amazon= pd.read_csv('AMZN_grouped_sentiment_double_singleday.csv') 
twitter_NLP_apple= pd.read_csv('AAPL_grouped_sentiment_double_singleday.csv') 
twitter_NLP_ibm= pd.read_csv('IBM_grouped_sentiment_double_singleday.csv') 
twitter_NLP_microsoft= pd.read_csv('MSFT_grouped_sentiment_double_singleday.csv') 
twitter_NLP_tesla= pd.read_csv('TSLA_grouped_sentiment_double_singleday.csv') 
#combining NLP files
twitter_NLP_list=[twitter_NLP_apple,twitter_NLP_amazon,twitter_NLP_ibm,twitter_NLP_microsoft,twitter_NLP_tesla]
twitter_NLP_combined = pd.concat(twitter_NLP_list, axis=0)
twitter_NLP_combined['Date'] = pd.to_datetime(twitter_NLP_combined['Date'])
twitter_NLP_combined=twitter_NLP_combined.reset_index(drop=True)
twitter_NLP_combined

,Date,polarity,Symbol
0,2021-09-25,0.500000,Apple
1,2021-09-25,0.500000,Apple
2,2021-09-25,0.000000,Apple
3,2021-09-25,0.100000,Apple
4,2021-09-25,0.000000,Apple
...,...,...,...
61866,2021-03-26,0.500000,Tesla
61867,2021-03-26,-0.025000,Tesla
61868,2021-03-26,0.000000,Tesla
61869,2021-03-26,0.220000,Tesla


In [ ]:
#merging twitter NLP data and stock data on columns Date and Symbol
df_merged = pd.merge(twitter_NLP_combined, stock_data, on=['Date','Symbol'], how='inner')
df_merged.rename(columns = {'polarity':'Polarity'}, inplace = True)

In [ ]:
#selecting only the wanted columns
df_merged_final=df_merged[['Date','lab_sameday','Polarity']]
df_merged_final

,Date,lab_sameday,Polarity
0,2021-09-24,1,0.166667
1,2021-09-24,1,0.000000
2,2021-09-24,1,0.375000
3,2021-09-24,1,0.000000
4,2021-09-24,1,0.234091
...,...,...,...
61690,2021-03-26,0,0.500000
61691,2021-03-26,0,-0.025000
61692,2021-03-26,0,0.000000
61693,2021-03-26,0,0.220000


In [ ]:
#selecting only the wanted columns
df_merged_final1=df_merged[['Date','lab_sameday','Polarity','Open','Close','Volume','Low','High']]
df_merged_final1

,Date,lab_sameday,Polarity,Open,Close,Volume,Low,High
0,2021-09-24,1,0.166667,145.66,146.92,53477870.0,145.56,147.4701
1,2021-09-24,1,0.000000,145.66,146.92,53477870.0,145.56,147.4701
2,2021-09-24,1,0.375000,145.66,146.92,53477870.0,145.56,147.4701
3,2021-09-24,1,0.000000,145.66,146.92,53477870.0,145.56,147.4701
4,2021-09-24,1,0.234091,145.66,146.92,53477870.0,145.56,147.4701
...,...,...,...,...,...,...,...,...
61690,2021-03-26,0,0.500000,641.87,618.71,33852830.0,599.89,643.8200
61691,2021-03-26,0,-0.025000,641.87,618.71,33852830.0,599.89,643.8200
61692,2021-03-26,0,0.000000,641.87,618.71,33852830.0,599.89,643.8200
61693,2021-03-26,0,0.220000,641.87,618.71,33852830.0,599.89,643.8200


In [ ]:
#df_merged_final.to_csv('/content/drive/MyDrive/stock_data_mine/labelled_dataset_sentiment_sameday_deployment_singleday.csv', index=False)

In [ ]:
#df_merged_final1.to_csv('/content/drive/MyDrive/stock_data_mine/labelled_dataset_sentiment_sameday_deployment_singleday_full.csv', index=False)

In [1]:
df_merged_final1.to_csv('labelled_dataset_full.csv', index=False)

NameError: ignored